In [1]:
#export
import torch as _torch, torch.nn as _nn, numpy as _np, math as _math
from functools import partial as _partial

In [2]:
#export
class Object:
    def __init__(self): self._defaultValueGenerator = None; self.repr = None
    @staticmethod
    def fromDict(_dict):
        """Creates an object with attributes from a dictionary"""
        answer = Object(); answer.__dict__.update(_dict); return answer
    @property
    def state(self):
        """Filtered __dict__"""
        answer = dict(self.__dict__); del answer["_defaultValueGenerator"]
        del answer["repr"]; answer.pop("getdoc", None); return answer
    def withAutoDeclare(self, defaultValueGenerator):
        """Sets this Object up so that if a field doesn't
        exist, it will automatically create it with a
        default value."""
        self._defaultValueGenerator = defaultValueGenerator; return self
    def __getitem__(self, idx): return getattr(self, idx)
    def __setitem__(self, idx, value): setattr(self, idx, value)
    def __contains__(self, item): return item in self.__dict__
    def __getattr__(self, attr):
        if attr.startswith("_"): raise AttributeError()
        if self._defaultValueGenerator != None:
            self.__dict__[attr] = self._defaultValueGenerator()
            return self.__dict__[attr]
        raise AttributeError
    def withRepr(self, _repr): self.repr = _repr; return self
    def __repr__(self):
        _dict = "\n".join([f"- {k}" for k in self.state.keys()])
        return self.repr or f"{type(self)}, with attrs:\n{_dict}"

In [3]:
a = Object().withAutoDeclare(lambda: 0)
a.mins += 10; a.mins += 3; assert a.mins == 13; assert a["mins"] == 13
a["maxs"] += 2; a.maxs += 36; assert a.maxs == 38; assert a["maxs"] == 38
assert a.state == {"mins": 13, "maxs": 38}

In [4]:
#export
class CancelRunException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass
def close(a, b):
    """Tests if 2 values are really close to each other"""
    test(_torch.allclose(_torch.tensor(a), _torch.tensor(b)))
def textToHtml(text:str) -> str:
    return text.replace("\n", "<br>").replace(" ", "&nbsp;")
def clearLine():
    """Prints a character that clears the current line"""
    print("\r" + " "*80 + "\r", end="")
def tab(text:str) -> str:
    """Adds a tab before each line"""
    return "\n".join(["    " + line for line in text.split("\n")])
def isNumeric(x) -> bool:
    """Returns whether object is actually a number"""
    return isinstance(x, (int, float, _np.float, _np.int))
def patch(_class:type, name=None, docs=None):
    """Patches a function to a class/object.
Args:
    _class: object to patch function. Can also be a type
    name (optional): name of patched function, if different from current
    docs (optional): docs of patched function. Can be object with __doc__ attr"""
    def inner(_function):
        _docs = docs
        if _docs != None and type(_docs) is not str: _docs = _docs.__doc__
        _function.__doc__ = _docs or _function.__doc__ or _class.__doc__
        setattr(_class, name or _function.__qualname__, _function)
        return _function
    return inner
def squeeze(_list):
    """If list only has 1 element, rethrn that element, else return original list"""
    if isinstance(_list, (tuple, list)) and len(_list) == 1: return _list[0]
    return _list
def raiseEx(ex):
    """Raises a specific exception. Useful in lambdas"""
    raise ex
def smooth(arr, consecutives=5):
    """Smoothes out array, so that y values are averages of the neighbors"""
    answer = []; s = 0
    for i, elem in enumerate(arr):
        s += elem
        if (i + 1) % consecutives == 0:
            answer.append(s / consecutives); s = 0
    return answer
def numDigits(num): return len(f"{num}")
def limitLines(s, limit=10):
    """If input string is too long, truncates it and adds ellipsis"""
    if len(splits := s.split("\n")) > limit:
        return "\n".join(splits[:limit]) + "\n....."
    else: return s

In [5]:
#export
class Range:
    def __init__(self, start=0, stop=None):
        if (isNumeric(start) and isNumeric(stop)):
            self.start, self.stop = start, stop
        elif isNumeric(start) and stop == None:
            self.start, self.stop = 0, start
        elif isinstance(start, (range, slice, Range)):
            self.start, self.stop = start.start, start.stop
        elif isinstance(start, (list, tuple)):
            self.start, self.stop = start[0], start[-1]
        else: raise Exception(f"Don't understand {start} and {stop}")
        self.delta = self.stop - self.start
    def __getitem__(self, index):
        if index == 0: return self.start
        if index == 1: return self.stop
        if type(index) == slice:
            return Range(range(self.start, self.stop)[index])
        raise Exception(f"Can't get index {index} of range [{self.start}, {self.stop}]")
    def _common(self, x, f):
        if isNumeric(x): return f(x)
        if isinstance(x, (list, tuple)):
            return [self._common(elem, f) for elem in x]
        if isinstance(x, (range, slice, Range)):
            return Range(self._common(x.start if x.start != None else 0, f), self._common(x.stop if x.stop != None else 1, f))
        raise Exception(f"Doesn't understand {x}")
    def __iter__(self): yield self.start; yield self.stop
    def toUnit(self, x):
        def f(self, x):
            if self.delta == 0: return float("nan")
            return (x - self.start) / self.delta
        return self._common(x, _partial(f, self))
    def fromUnit(self, x):
        def f(self, x): return x * self.delta + self.start
        return self._common(x, _partial(f, self))
    def toRange(self, _range, x):
        def f(self, _range, x): return Range(_range).fromUnit(self.toUnit(x))
        return self._common(x, _partial(f, self, _range))
    @property
    def slice(self): return slice(_math.floor(self.start+0.001), _math.floor(self.stop+0.001))
    @property
    def range(self): return range(_math.floor(self.start+0.001), _math.floor(self.stop+0.001))
    @staticmethod
    def proportionalSlice(r1, r2, r1Slice:slice):
        r1, r2 = Range(r1), Range(r2)
        ar2 = r1.toRange(r2, (ar1 := r1[r1Slice]))
        return ar1, ar2
    def __str__(self): return f"[{self.start}, {self.stop}]"
    def __eq__(self, _range):
        _range = Range(_range)
        return abs(_range.start - self.start) < 1e-9 and abs(_range.stop - self.stop) < 1e-9
    def __repr__(self):
        return f"""A range of numbers: [{self.start}, {self.stop}]. Can do:
- r.toUnit(x): will convert x from range [{self.start}, {self.stop}] to [0, 1]
- r.fromUnit(x): will convert x from range [0, 1] to range [{self.start}, {self.stop}]
- r.toRange([a, b], x): will convert x from range [{self.start}, {self.stop}] to range [a, b]
- r[0], r[1], r.start, r.stop: get start and stop values of range

Note: for conversion methods, you can pass in"""

In [6]:
def assertRange(_range, start, stop):
    assert _range.start == start
    assert _range.stop == stop
assertRange(Range(range(5)), 0, 5)
assert Range(-2, 2).toUnit(0) == 0.5
assert Range(10).toUnit([5, 7]) == [0.5, 0.7]
assert Range(10).toUnit(slice(5, 7)) == Range(0.5, 0.7)
assert Range(10).fromUnit(slice(0.7)) == Range(0, 7)
assert Range(10).toRange([0, 100], [5, 7]) == [50, 70]
assert list(range(-5, 100)[Range(3).slice]) == [-5, -4, -3]
assert list(Range(5).range) == [0, 1, 2, 3, 4]
a, b = Range.proportionalSlice(Range(10), Range(100), slice(2, 5))
assert a.range == range(2, 5)
assert b.range == range(20, 50)
assert [*Range(2, 5)] == [2, 5]
assert Range(10).fromUnit(slice(0.2, 0.5)).range == range(2, 5)

In [7]:
#export
def polyfit(x, y, deg):
    params = _np.polyfit(x, y, deg)
    def _inner(_x):
        answer = _np.zeros_like(_x, dtype=_np.float)
        for expo, param in enumerate(params):
            answer += param * _x**(len(params)-expo-1)
        return answer
    return _inner
def derivative(f, delta=1e-3):
    return lambda x: (f(x + delta) - f(x)) / delta
def optimize(f, v=1, threshold=1e-5):
    fD = derivative(f)
    while abs(f(v)) > threshold: v = v - f(v)/fD(v)
    return v

In [8]:
#export
@patch(_nn.Module, name="importParams")
def _importParams(self:_nn.Module, params:list):
    for oldParam, newParam in zip(self.parameters(), params):
        oldParam.data = newParam.data.clone()
@patch(_nn.Module, name="exportParams")
def _exportParams(self:_nn.Module):
    return [param.data.clone() for param in self.parameters()]
@patch(_nn.Module, name="getParamsVector")
def _getParamsVector(model:_nn.Module):
    answer = []
    for param in model.parameters():
        a = _torch.randn(param.shape).to(param.device)
        b = (param.std() if len(param.squeeze().shape) > 0 else 1)
        answer.append(a * b)
    return answer

In [9]:
!../export.py _basics

Current dir: /home/kelvin/repos/labs/k1lib, ../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.0
Uninstalling k1lib-0.1.0:
  Successfully uninstalled k1lib-0.1.0
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/callbacks.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1lib/_basics.py -> build/lib/k1lib
copying k1lib/nn.py -> bui